In [1]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_mescladas(conn, test_prontuario):
    """Get mescladas data for a specific patient"""
    query = f"""
        SELECT * FROM silver.mesclada_vendas
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)

        mescladas_df = get_mescladas(conn, test_prontuario)
        return timeline_df, finops_df, mescladas_df
        
    finally:
        conn.close()

In [ ]:
if __name__ == "__main__":

    ## casos para testes iniciais

    # patient_id = 825_890 # Claudia
    # patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    # patient_id = 162_173 # caso bonito: 5 ciclos - 5 pagamentos

    ## resolvidos
    # patient_id = 641_950 # 11 cycles without payment
    # patient_id = 150_491 # ovo recepção
    # patient_id = 155_282 # medico externo
    # patient_id = 738_349 # (marido 738_350 não aparece no clinisys) -- sem pagamento
    # patient_id = 839574 # utero de substituicao (815299)
    # patient_id = 842663 # doadora
    # patient_id = 678_207 # numero cliente duplicado no diario_vendas (78207) - BH
    # patient_id = 831_609 # pagamento no ID errado do paciente, mas certo no marido (CLiente = 833_788)
    # patient_id = 662_298 # paciente 50347 em mescladas, cliente 62298 no diario - BH
    
    # patient_id = 162733 # Beshaatova - CLIENTE/PACIENTE 64645 - Filipe
    # patient_id = 832_556 # utero de substituição - pago por 831_609 - Filipe
    # patient_id = 881_426 # utero de substituição - 126881 - Filipe

    ## abertos

    # patient_id = 880_484 # erro de migração de cadastro - pago  cliente 55623
    # patient_id = 867_007 # pago - nomes diferentes no clinisys e mescladas (erro de cadastro no Totvs)
    # patient_id = 875_610 # autorizado pela gerencia, sem custo
    # patient_id = 178125 # pago em 2021, primeira transferencia em 2025

    ## sem pagamento ou sem faturamento
    # patient_id = 166_697 # FOT R$ 4695 em 11/06/2025 + biópisia
    # patient_id = 172_179 # funcionaria 
    # patient_id = 885_517  # Matheus Roque - pago na Mater Prime
    # patient_id = 879_641 # Matheus Roque - pago na Mater Prime
    # patient_id = 887_035 # relato que está pago no orçamento, mas não lançado e não achamos NF
    # patient_id = 873_039 # não pago - FET
    


    patient_id = 878_381 



    timeline_df, finops_df, mescladas_df = main(patient_id)


Creating timeline for patient: 878381
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [60]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,878381.0,1,0,1,0.0,0.0,2025-06-03,2025-06-03,NaT,NaT,878381.0,0,1,1,0,0,1. HTT SP - Ibirapuera


In [61]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,878381.0,1436533,2025-07-10,extrato_atendimentos,US - Obstétrico,None,5. HTT Belo Horizonte,None,False,{}
1,878381.0,1436463,2025-07-01,extrato_atendimentos,Retorno Psicologia,None,5. HTT Belo Horizonte,None,False,{}
2,878381.0,1431637,2025-06-30,extrato_atendimentos,1º US Obstétrico: Confirmação,None,1. HTT SP - Ibirapuera,None,False,{}
3,878381.0,1426822,2025-06-23,extrato_atendimentos,Acompanhamento Beta HCG,None,1. HTT SP - Ibirapuera,None,False,{}
4,878381.0,1424972,2025-06-16,extrato_atendimentos,Acompanhamento Beta HCG,None,1. HTT SP - Ibirapuera,None,False,{}
5,878381.0,1415906,2025-06-13,extrato_atendimentos,Acompanhamento Beta HCG,None,1. HTT SP - Ibirapuera,None,False,{}
6,878381.0,34602,2025-06-03,tratamentos,Ciclo de Congelados,2,1. HTT SP - Ibirapuera,Gestação Clínica,False,{}
7,878381.0,17904,2025-06-03,descongelamentos_embrioes,-,None,None,None,False,"{""CodDescongelamento"": ""1153/25"", ""Unidade"": ""1""}"
8,878381.0,1414694,2025-06-03,extrato_atendimentos,FET: Excedente Próprio,None,1. HTT SP - Ibirapuera,None,False,{}
9,878381.0,1410868,2025-06-02,extrato_atendimentos,Resultado Exames,None,1. HTT SP - Ibirapuera,None,False,{}


In [62]:
mescladas_df.sort_values(by='DT Emissao', ascending=False)

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,878381,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-09-08,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,31006,RPS,30884,0,CRIOPRESERVACAO,892266,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,2610,20250916_153309,Mescladas2.xlsx
4,878381,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-08-08,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,25575,RPS,25439,0,CRIOPRESERVACAO,892266,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,392669,20250916_153309,Mescladas2.xlsx
3,878381,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-07-08,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,20043,RPS,19951,0,CRIOPRESERVACAO,892266,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,386332,20250916_153309,Mescladas2.xlsx
2,878381,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-06-09,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,14742,RPS,14664,0,CRIOPRESERVACAO,892266,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,380494,20250916_153309,Mescladas2.xlsx
1,878381,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-05-08,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,9857,RPS,9807,0,CRIOPRESERVACAO,892266,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,374882,20250916_153309,Mescladas2.xlsx
5,<NA>,JESSICA CARVAL,878381,JESSICA CARVALHO DE OLIVEIRA,878381,2025-04-03,"OVIDREL 0,5ML",1.0,555.0,Estimulação,101,502,254441,RPS,202500000008603,16,MEDICOS EXTERNOS,878381,,200143,0.0,0.0,0.0,0.0,Belo Horizonte,0,0,11310100005,,,Estimulação,0,0,5,MEDICAMENTO,None,NaT,False,580633,20250916_153309,Mescladas2.xlsx
